In [2]:
import pandas as pd
import numpy as np
import urllib.request as ul
import networkx as nx
import time, math, sys, re, random, copy, glob, pickle


In [42]:
def gen_FS_RS_cij(arcs, nodes):
    j=0
    forward = [[] for _ in range(nodes)]
    reverse = [[] for _ in range(nodes)]
    
    for i in arcs:
        forward[i[0]].append(i[1])
        reverse[i[1]].append(i[0])

    # Create dictionary of nodes accessible from other nodes
    FS = {n:(forward[n]) for n in range(nodes)}
    RS = {n:(reverse[n]) for n in range(nodes)}

    cij={tuple(arcs[i][0:2]):arcs[i][2] for i in range(len(arcs))}
    
    return FS, RS, cij

In [ ]:
def import_f(fname):
    with open(fname,'r') as f:
        
        # read initial line with number of nodes, arcs
        nodes = int(f.readline().split()[0])
        arcs = []
        l = 0
        
        for i in f:
            # skip node b-values for min cost flow
            if l >= nodes:
                # 
                arc_value = i.split()
                # append arc to list in tuple (head node, tail node, arc cost)
                arcs.append((int(arc_value[0]),int(arc_value[1]),float(arc_value[2])))
            l+=1
            
    return nodes, arcs

In [43]:
def single_source_dijkstra_naive(nodes,arcs,source=0):
    
    # Generate forward star, reverse star, arc cost dictionary from arc list
    FS, RS, cij = gen_FS_RS_cij(arcs,nodes)
    
    # Initialize DataFame - rows are nodes, columns are distance labels, previous label,
    # and labeled status (True/False)
    d = [np.inf for i in range(nodes)]
    l = pd.DataFrame(d)
    l.columns = ['distance']
    l['labeled'] = False
    l['pred'] = -1
    
    # Set initial 
    l.at[source,'distance']=0
    
    # Begin iterations - continue until all labels are set
    while l['labeled'].sum() < nodes:
        
        # Find smallest unlabeled node, set as i
        i = l.loc[l['labeled']==False]['distance'].idxmin()
        
        # Update label for min node 
        l.at[i,'labeled']=True
        
        # Retrieve distance label
        d_i = l.at[i,'distance']
        
        # Update arcs in forward star of node i
        for j in FS[i]:
            
            # Retrieve current distance label
            d_j = l.at[j,'distance']
            
            # Compare distance labels in forward star of node i
            if d_j > (cij[(i,j)] + d_i):
                l.at[j,'distance'] = (cij[(i,j)] + d_i)
                l.at[j,'pred'] = i

    return l

In [44]:
def single_source_dijkstra_d_heap(nodes,arcs,source=0,d=2):
    
    # Functions for heap management
    
    def insert(i, heap):
        heap.append(i)
        heap = sift_up(i,heap)
        
        return heap
    
    def delete_min(heap):
        
        heap = swap(heap[0],heap[-1],heap)
        heap.pop(-1)
        
        heap = sift_down(heap[0],heap)
            
        return heap
    
    def sift_up(i, heap):
        ix_i = heap.index(i)
        ix_parent = math.ceil((ix_i)/d)-1
        parent = heap[ix_parent]
    
        d_i = l.at[i,'distance']
        d_parent = l.at[parent,'distance']
        
        # Iterate until node i is root, or heap order is restored
        while ix_i > 0 and d_i < d_parent:
        
            # Swap node i and parent
            heap = swap(i,parent, heap)
            ix_i = heap.index(i)

            # Update new parent information
            # Subtracting one due to 0 index of python list
            ix_parent = math.ceil((ix_i)/d)-1
            parent = heap[ix_parent]
            d_parent = l.at[parent,'distance']
        
        return heap
    
    def sift_down(i,heap):
        ix_i = heap.index(i)
        d_i = l.at[i,'distance']
        
        children, d_children, min_child, ix_min_child = find_children(i,heap)
        
        # Iterate while 
        while len(children) > 0 and d_i > d_children[ix_min_child]:
            
            heap = swap(i,min_child, heap)
            
            children, d_children, min_child, ix_min_child = find_children(i,heap)
            
        return heap
    
    def find_children(i,heap):
        ix_i = heap.index(i)
        # Identify first child index of node i - add 1 to index initially to account for zero index, then remove
        ix_first_child = (ix_i+1)*d-d+1
        
        # Identify last child index 
        ix_last_child = ix_first_child + d
        
        # Determine if node i is a leaf node and has no children
        if ix_first_child < len(heap):
            
            # Determine if children 
            if ix_last_child <= len(heap):
                children = heap[ix_first_child:(ix_last_child)]
            else:
                children = heap[ix_first_child:]
            
            # Create list of children distance, and find the min child
            d_children = [l.at[c,'distance'] for c in children]
            ix_min_child = min(range(len(d_children)), key=d_children.__getitem__)
            min_child = children[ix_min_child]
            
        else:
            children =[]
            d_children=[]
            min_child=-1
            ix_min_child=-1
        
        return children, d_children, min_child, ix_min_child
    
    def swap(i, j, heap):
        ix_i = heap.index(i)
        ix_j = heap.index(j)
        
        if len(heap) == 2:
            return heap[::-1]
        # Determin which node has the smaller key value
        if ix_i < ix_j:
            # Delete higher node label first
            del heap[ix_j]
            del heap[ix_i]
            # Insert lower node label first
            heap.insert(ix_i,j)
            heap.insert(ix_j,i)
        
        else:
            # Delete higher node label first
            del heap[ix_i]
            del heap[ix_j]
            # Insert lower node label first
            heap.insert(ix_j,i)
            heap.insert(ix_i,j)
        return heap
    
    # Start of implementation
    # Generate forward star, reverse star, arc cost dictionary from arc list
    FS, RS, cij = gen_FS_RS_cij(arcs,nodes)
    
    # Initialize list of 
    e = [np.inf for i in range(nodes)]
    l = pd.DataFrame(e)
    l.columns = ['distance']
    l['pred'] = -1
    
    # Set initial 
    l.at[source,'distance']=0
    
    # Generate initial heap with source
    heap = [source]
    
    
    # Begin iterations - continue until no more nodes in heap
    while len(heap) > 0:
        
        # Find smallest unlabeled node i and remove from heap
        i = heap[0]
        if len(heap) > 1:
            heap = delete_min(heap)
        else:
            heap.pop()
        
        # Update label for min node 
        #l.at[i,'labeled']=True
        
        # Retrieve distance label
        d_i = l.at[i,'distance']
        
        # Check all node labels in the forward star of i
        for j in FS[i]:
            
            val = cij[(i,j)] + d_i
            # Retrieve
            d_j = l.at[j,'distance']
            #print(d_j, val, (i,j))
            
            if d_j > val:
                l.at[j,'distance'] = val
                l.at[j,'pred'] = i
                if d_j == float('inf'):
                    heap = insert(j,heap)
                else:
                    heap = sift_up(j,heap)
    return l